# **ANÁLISE 1 - ENTREGA**

![](https://media.giphy.com/media/oenruB2DKC7p6/giphy.gif)

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from itertools import combinations
from scipy.stats import ttest_ind
from scipy import stats
import networkx as nx
import unidecode
import freeman as fm
import io
from collections import Counter
from numpy import median
import statistics 


# Loading the dataset

In [2]:
netflix_overall=pd.read_csv("netflix_titles.csv")

In [3]:
netflix_movies=netflix_overall[netflix_overall['type']=='Movie']

## Contagem do paises dos filmes

In [4]:
netflix_movies["country"].value_counts()

United States                                                      1482
India                                                               724
United Kingdom                                                      170
Canada                                                               88
Spain                                                                80
                                                                   ... 
China, India, Nepal                                                   1
Spain, United Kingdom, United States                                  1
Denmark, United Kingdom, South Africa, Sweden, Belgium                1
Australia, France                                                     1
France, Switzerland, Spain, United States, United Arab Emirates       1
Name: country, Length: 482, dtype: int64

## Contagem do total de atores 

In [5]:
netflix_movies = netflix_movies.dropna(axis=0)
cast = netflix_movies['cast']


actors = []
temp = []


for i in cast:
    temp = i.split(",")
    for name in temp:
        actors.append(name)
        
actors_unique = Counter(actors).keys()  
print("atores sem repetição:",len(actors_unique))
print("atores com repetição:", len(actors))
#Counter(actors).values()



atores sem repetição: 19519
atores com repetição: 28508


In [6]:
netflix_movies = netflix_movies.reset_index(drop=True)

### Dividir os filmes por ano

In [7]:
netflix_movies['added_2017'] = False
netflix_movies['added_2018'] = False
netflix_movies['added_2019'] = False
netflix_movies['added_3'] = False
for i in range(0, len(netflix_movies)):
    if (netflix_movies['date_added'][i][-4:] == '2017'):
        netflix_movies['added_2017'][i] = True
    else:
        netflix_movies['added_2017'][i] = False

for i in range(0, len(netflix_movies)):
    if (netflix_movies['date_added'][i][-4:] == '2018'):
        netflix_movies['added_2018'][i] = True
    else:
        netflix_movies['added_2018'][i] = False

for i in range(0, len(netflix_movies)):
    if (netflix_movies['date_added'][i][-4:] == '2019'):
        netflix_movies['added_2019'][i] = True
    else:
        netflix_movies['added_2019'][i] = False
        
for i in range(0, len(netflix_movies)):
    if (netflix_movies['date_added'][i][-4:] == '2017') or (netflix_movies['date_added'][i][-4:] == '2018') or (netflix_movies['date_added'][i][-4:] == '2019'):
        netflix_movies['added_3'][i] = True
    else:
        netflix_movies['added_3'][i] = False

<ipython-input-7-52d02a9510c0>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-52d02a9510c0>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-52d02a9510c0>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-52d02a9510c0>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [8]:
netflix_movies_2017 = netflix_movies[netflix_movies.added_2017 == True]
netflix_movies_2018 = netflix_movies[netflix_movies.added_2018 == True]
netflix_movies_2019 = netflix_movies[netflix_movies.added_2019 == True]
netflix_movies_3 = netflix_movies[netflix_movies.added_3 == True]

genre_2017 = netflix_movies_2017['listed_in']
genre_count_2017 = pd.Series(dict(Counter(','.join(genre_2017).replace(' ,',',').replace(', ',',')
                                    .split(',')))).sort_values(ascending=False)

genre_2018 = netflix_movies_2018['listed_in']
genre_count_2018 = pd.Series(dict(Counter(','.join(genre_2018).replace(' ,',',').replace(', ',',')
                                    .split(',')))).sort_values(ascending=False)

genre_2019 = netflix_movies_2019['listed_in']
genre_count_2019 = pd.Series(dict(Counter(','.join(genre_2019).replace(' ,',',').replace(', ',',')
                                    .split(',')))).sort_values(ascending=False)
genre_3 = netflix_movies_3['listed_in']
genre_count_3 = pd.Series(dict(Counter(','.join(genre_3).replace(' ,',',').replace(', ',',')
                                    .split(',')))).sort_values(ascending=False)

In [11]:
# GMP 

genre_count_3

International Movies        1620
Dramas                      1435
Comedies                     936
Action & Adventure           527
Independent Movies           489
Thrillers                    344
Romantic Movies              341
Children & Family Movies     264
Documentaries                261
Horror Movies                218
Stand-Up Comedy              187
Music & Musicals             175
Sci-Fi & Fantasy             153
Sports Movies                 97
Classic Movies                64
LGBTQ Movies                  48
Cult Movies                   43
Faith & Spirituality          39
Anime Features                36
Movies                        16
dtype: int64

In [15]:
sum3 = sum(genre_count_3)
genre_count_3 = genre_count_3.to_frame()
genre_count_3[0] = genre_count_3[0].astype(float)

In [19]:
for i in range(len(genre_count_3)):
    temp = genre_count_3[0][i]/sum3
    genre_count_3[0][i] = temp

In [20]:
actors_and_genres3 = netflix_movies_3.iloc[:, [4, 10]]

In [21]:
actors_and_genres3

,cast,listed_in
0,"Alan Marriott, Andrew Toth, Brian Dobson, Cole...","Children & Family Movies, Comedies"
1,"Nesta Cooper, Kate Walsh, John Michael Higgins...",Comedies
2,"Antonio Banderas, Dylan McDermott, Melanie Gri...","International Movies, Sci-Fi & Fantasy, Thrillers"
3,Fabrizio Copano,Stand-Up Comedy
4,"James Franco, Kate Hudson, Tom Wilkinson, Omar...","Action & Adventure, Thrillers"
...,...,...
3670,"Abbie Cornish, Diego Klattenhoff, Justin Long,...","Dramas, International Movies, Thrillers"
3671,"Shashi Kapoor, Sulakshana Pandit, Mehmood, Sud...","Action & Adventure, International Movies, Musi..."
3672,"Ana Polvorosa, Candela Peña, Macarena Gómez, C...","Dramas, International Movies"
3673,"Alan Sabbagh, Julieta Zylberberg, Usher Barilk...","Dramas, International Movies"


### Selecionar apenas gênero e atores por ano

In [28]:
network_3_temp_actor = pd.concat([pd.Series(row['cast'], row['listed_in'].split(','))              
                    for _, row in actors_and_genres3.iterrows()]).reset_index()

network_3_temp_actor = network_3_temp_actor.rename(columns={'index': 'categoria', 0: 'ator'})

network_3_actor = pd.concat([pd.Series(row["categoria"], row['ator'].split(','))              
                    for _, row in network_3_temp_actor.iterrows()]).reset_index()

network_3_actor = network_3_actor.rename(columns={'index': 'ator', 0: 'categoria'})

In [29]:
actor_genre_3 = network_3_actor.groupby('ator')['categoria'].apply(list).reset_index(name='genero') 

In [36]:
## GA

actor_3 = netflix_movies_3['cast']
actor_count_3 = pd.Series(dict(Counter(','.join(actor_3).replace(' ,',',').replace(', ',',')
                                    .split(',')))).sort_values(ascending=False)

actor_count_3 = actor_count_3.to_frame()

actor_count_3.reset_index(level = 0, inplace = True)
actor_count_3 = actor_count_3.rename(columns={'index': 'ator', 0: 'GA'})

actor_count_3['EA'] = actor_count_3['GA'].apply(lambda x: 1/x)

In [37]:
network_3_temp = pd.concat([pd.Series(row['listed_in'], row['cast'].split(','))              
                    for _, row in actors_and_genres3.iterrows()]).reset_index()

network_3_temp = network_3_temp.rename(columns={'index': 'ator', 0: 'categoria'})

network_3 = pd.concat([pd.Series(row["ator"], row['categoria'].split(','))              
                    for _, row in network_3_temp.iterrows()]).reset_index()

network_3 = network_3.rename(columns={'index': 'categoria', 0: 'ator'})

### Limpando caractéres especiais

In [41]:
for i in range(len(network_3['ator'])):
    if "'" in network_3['ator'][i]:
        network_3['ator'][i] = network_3['ator'][i].replace("'", "")
    if '"' in network_3['ator'][i]:
        network_3['ator'][i] = network_3['ator'][i].replace('"', "")
        
    network_3['ator'][i] = unidecode.unidecode(network_3['ator'][i])

### Código para criar o arquivo GML - 2017/2018/2019

In [42]:
nodes1_3 = list(set(network_3['ator']))
nodes2_3 = list(set(network_3['categoria']))

temp_cat_3 = []
temp_ator_3 = []
temp_ator2_3 = []
edgeCheck_3 = {}
listweights_3 = []

with io.open("./network2_3.gml", "w") as f:
    f.write('graph [\n')
    f.write('  directed 0\n')

    for i in range(len(network_3['ator'])):
        if not network_3['ator'][i] in temp_ator_3:
            f.write('  node [\n')
            f.write('    id "{}"\n'.format(network_3['ator'][i]))
            f.write('  ]\n')
            temp_ator_3.append(network_3['ator'][i])
        
    for i in range(len(network_3['categoria'])):
        if network_3['categoria'][i][0] == ' ':
            network_3['categoria'][i] = network_3['categoria'][i][1:]
        if not network_3['categoria'][i] in temp_cat_3:
            f.write('  node [\n')
            f.write('    id "{}"\n'.format(network_3['categoria'][i]))
            f.write('  ]\n')
            temp_cat_3.append(network_3['categoria'][i])
            
    for index, row in network_3.iterrows():
        if row['categoria'][0] == ' ':
            keytemp = row['ator'] + ',' + row['categoria'][1:]
        else:
            keytemp = row['ator'] + ',' + row['categoria']
        
        if keytemp in edgeCheck_3:
            edgeCheck_3[keytemp] += 1
        else:
            edgeCheck_3[keytemp] = 1
            
    for key in edgeCheck_3:
        ator, categoria = key.split(',')
        weight = edgeCheck_3[key]
        if weight > 1:
            temp_ator2_3.append(ator)
            listweights_3.append(weight)
        
        f.write('  edge [\n')
        f.write('    source "{}"\n'.format(ator))
        f.write('    target "{}"\n'.format(categoria))
        f.write('    weight {}\n'.format(weight))
        f.write('  ]\n')  

    f.write(']\n')
        

In [43]:
g_3 = fm.load('network2_3.gml')

## Transformando em uma one-mode

In [44]:
g1_3 = fm.Graph(nx.Graph())

In [45]:
for n in temp_ator2_3:
    g1_3.add_node(n)

In [46]:
for n, m in combinations(g1_3.nodes, 2):

    # Muito cuidado para não usar g1 aqui!
    # Estamos analisando os vizinhos em g2.
    repos_n = set(g_3.neighbors(n))
    repos_m = set(g_3.neighbors(m))

    # Em sets é fácil calcular intersecção.
    weight = len(repos_n & repos_m)

    # Adicionamos só se weight for positivo.
    if weight > 0:
        g1_3.add_edge(n, m)
        g1_3.edges[n, m]['weight'] = weight

In [47]:
isolated_3 = [n for n in g1_3.nodes if g1_3.degree(n) == 0]

for n in isolated_3:
    g1_3.remove_node(n)

In [48]:
weights_3 = [g1_3.edges[n, m]['weight'] for n, m in g1_3.edges]
nx.write_gml(g1_3,'one_mode.gml')

## Métricas - 2017/2018/2019

In [50]:
# CA (r) 

cluster_3 = nx.clustering(g1_3)

In [51]:
edgeCheck_genero_3 = {}

for key in edgeCheck_3:
    ator, categoria = key.split(',')
    if ator in cluster_3.keys():
        edgeCheck_genero_3[key] = cluster_3[ator]

In [52]:
cluster_international_3 = []
cluster_drama_3 = []
cluster_comedies_3 = []
cluster_independent_3 = []
cluster_action_3 = []
cluster_documentaries_3 = []
cluster_thrillers_3 = []
cluster_romantic_3 = []
cluster_children_3 = []
cluster_stand_up_3 = []
cluster_music_3 = []
cluster_horror_3 = []
cluster_sports_3 = []
cluster_scifi_3 = []
cluster_anime_3 = []
cluster_LGBTQ_3 = []
cluster_classic_3 = []
cluster_cult_3 = []
cluster_faith_3 = []
cluster_movies_3 = []

for key in edgeCheck_genero_3:
    ator, categoria = key.split(',')
    if categoria == genre_count_3[0].keys()[0]:
        cluster_international_3.append(edgeCheck_genero_3[key])
        
    elif categoria == genre_count_3[0].keys()[1]:
        cluster_drama_3.append(edgeCheck_genero_3[key]) 
        
    elif categoria == genre_count_3[0].keys()[2]:
        cluster_comedies_3.append(edgeCheck_genero_3[key])
        
    elif categoria == genre_count_3[0].keys()[3]:
        cluster_independent_3.append(edgeCheck_genero_3[key]) 
        
    elif categoria == genre_count_3[0].keys()[4]:
        cluster_action_3.append(edgeCheck_genero_3[key])
        
    elif categoria == genre_count_3[0].keys()[5]:
        cluster_documentaries_3.append(edgeCheck_genero_3[key])
        
    elif categoria == genre_count_3[0].keys()[6]:
        cluster_thrillers_3.append(edgeCheck_genero_3[key])
        
    elif categoria == genre_count_3[0].keys()[7]:
        cluster_romantic_3.append(edgeCheck_genero_3[key])
 
    elif categoria == genre_count_3[0].keys()[8]:
        cluster_children_3.append(edgeCheck_genero_3[key]) 
        
    elif categoria == genre_count_3[0].keys()[9]:
        cluster_stand_up_3.append(edgeCheck_genero_3[key])
        
    elif categoria == genre_count_3[0].keys()[10]:
        cluster_music_3.append(edgeCheck_genero_3[key]) 
        
    elif categoria == genre_count_3[0].keys()[11]:
        cluster_horror_3.append(edgeCheck_genero_3[key])
        
    elif categoria == genre_count_3[0].keys()[12]:
        cluster_sports_3.append(edgeCheck_genero_3[key])
        
    elif categoria == genre_count_3[0].keys()[13]:
        cluster_scifi_3.append(edgeCheck_genero_3[key])
        
    elif categoria == genre_count_3[0].keys()[14]:
        cluster_anime_3.append(edgeCheck_genero_3[key])
        
    elif categoria == genre_count_3[0].keys()[15]:
        cluster_LGBTQ_3.append(edgeCheck_genero_3[key]) 
        
    elif categoria == genre_count_3[0].keys()[16]:
        cluster_classic_3.append(edgeCheck_genero_3[key])
        
    elif categoria == genre_count_3[0].keys()[17]:
        cluster_cult_3.append(edgeCheck_genero_3[key]) 
        
    elif categoria == genre_count_3[0].keys()[18]:
        cluster_faith_3.append(edgeCheck_genero_3[key])
        
    elif categoria == genre_count_3[0].keys()[19]:
        cluster_movies_3.append(edgeCheck_genero_3[key])
    
    else:
        print("deu errado", key)

In [53]:
cluster_international_mean_3 = statistics.mean(cluster_international_3)
cluster_drama_mean_3 = statistics.mean(cluster_drama_3)
cluster_comedies_mean_3 = statistics.mean(cluster_comedies_3)
cluster_independent_mean_3 = statistics.mean(cluster_independent_3)
cluster_action_mean_3 = statistics.mean(cluster_action_3)
cluster_documentaries_mean_3 = statistics.mean(cluster_documentaries_3)
cluster_thrillers_mean_3 = statistics.mean(cluster_thrillers_3)
cluster_romantic_mean_3 = statistics.mean(cluster_romantic_3)
cluster_children_mean_3 = statistics.mean(cluster_children_3)
cluster_stand_up_mean_3 = 0
# = statistics.mean(cluster_stand_up)
cluster_music_mean_3 = statistics.mean(cluster_music_3)
cluster_horror_mean_3 = statistics.mean(cluster_horror_3)
cluster_sports_mean_3 = statistics.mean(cluster_sports_3)
cluster_scifi_mean_3 = statistics.mean(cluster_scifi_3)
cluster_anime_mean_3 = statistics.mean(cluster_anime_3)
cluster_LGBTQ_mean_3 = statistics.mean(cluster_LGBTQ_3)
cluster_classic_mean_3 = statistics.mean(cluster_classic_3)
cluster_cult_mean_3 = statistics.mean(cluster_cult_3)
cluster_faith_mean_3 = statistics.mean(cluster_faith_3)
cluster_movies_mean_3 = statistics.mean(cluster_movies_3)

### Regressão Linear entre a média dos cluster e a popularidade dos gêneros - 2017/2018/2019

In [54]:
cluster_mean_3 = []
cluster_mean_3.extend((cluster_international_mean_3, cluster_drama_mean_3,cluster_comedies_mean_3,
                          cluster_independent_mean_3, cluster_action_mean_3, cluster_documentaries_mean_3,
                          cluster_thrillers_mean_3, cluster_romantic_mean_3, cluster_children_mean_3, 
                          cluster_stand_up_mean_3, cluster_music_mean_3, cluster_horror_mean_3, 
                          cluster_sports_mean_3, cluster_scifi_mean_3, cluster_anime_mean_3, cluster_LGBTQ_mean_3, 
                          cluster_classic_mean_3, cluster_cult_mean_3, cluster_faith_mean_3, cluster_movies_mean_3))

movies_count_3 = []
for i in range(0, len(genre_count_3[0])):
    movies_count_3.append(genre_count_3[0][i])

In [58]:
lin_reg_3 = sm.OLS(cluster_mean_3, movies_count_3)
result_3 = lin_reg_3.fit()
result_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.396
Model:                            OLS   Adj. R-squared (uncentered):              0.364
Method:                 Least Squares   F-statistic:                              12.45
Date:                Mon, 23 Nov 2020   Prob (F-statistic):                     0.00225
Time:                        13:23:14   Log-Likelihood:                         -21.268
No. Observations:                  20   AIC:                                      44.54
Df Residuals:                      19   BIC:                                      45.53
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             7.1978      2.040      3.528      0.002       2.928      11.468
==============================================================================
Omnibus:                        8.858   Durbin-Watson:                   0.227
Prob(Omnibus):                  0.012   Jarque-Bera (JB):                6.708
Skew:                          -1.382   Prob(JB):                       0.0349
Kurtosis:                       3.637   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Teste de hipótese (teste-t) - 2017/2018/2019

In [59]:
def rSubset(arr, r): 
    # return list of all subsets of length r 
    # to deal with duplicate subsets use 
    # set(list(combinations(arr, r))) 
    return list(combinations(arr, r)) 

In [60]:
arr_3 = [cluster_international_3,cluster_drama_3,cluster_comedies_3,cluster_independent_3,cluster_action_3,
            cluster_documentaries_3,cluster_thrillers_3,cluster_romantic_3,cluster_children_3,cluster_stand_up_3,
            cluster_music_3,cluster_horror_3,cluster_sports_3,cluster_scifi_3,cluster_anime_3,cluster_LGBTQ_3,
            cluster_classic_3,cluster_cult_3,cluster_faith_3,cluster_movies_3]

indexes_3 = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]

dupla_indexes_3 = rSubset(indexes_3, 2)
p_valor_bom_3 = 0
p_valor_ruim_3 = 0

for i in range(len(dupla_indexes_3)):
    if ttest_ind(arr_3[dupla_indexes_3[i][0]], arr_3[dupla_indexes_3[i][1]])[1] <= 0.05:
        print("Duplas com P-valor bom: ", genre_count_3[0].keys()[dupla_indexes_3[i][0]], "e", genre_count_3[0].keys()[dupla_indexes_3[i][1]])
        p_valor_bom_3 += 1
    else:
        print("Duplas com P-valor ruim: ", genre_count_3[0].keys()[dupla_indexes_3[i][0]], "e", genre_count_3[0].keys()[dupla_indexes_3[i][1]])
        p_valor_ruim_3 +=1
        
print(p_valor_bom_3)
print(p_valor_ruim_3)

Duplas com P-valor bom:  International Movies e Dramas
Duplas com P-valor bom:  International Movies e Comedies
Duplas com P-valor bom:  International Movies e Action & Adventure
Duplas com P-valor bom:  International Movies e Independent Movies
Duplas com P-valor ruim:  International Movies e Thrillers
Duplas com P-valor bom:  International Movies e Romantic Movies
Duplas com P-valor bom:  International Movies e Children & Family Movies
Duplas com P-valor bom:  International Movies e Documentaries
Duplas com P-valor ruim:  International Movies e Horror Movies
Duplas com P-valor bom:  International Movies e Stand-Up Comedy
Duplas com P-valor bom:  International Movies e Music & Musicals
Duplas com P-valor bom:  International Movies e Sci-Fi & Fantasy
Duplas com P-valor ruim:  International Movies e Sports Movies
Duplas com P-valor bom:  International Movies e Classic Movies
Duplas com P-valor bom:  International Movies e LGBTQ Movies
Duplas com P-valor bom:  International Movies e Cult

Duplas com P-valor bom:  Sports Movies e Classic Movies
Duplas com P-valor bom:  Sports Movies e LGBTQ Movies
Duplas com P-valor bom:  Sports Movies e Cult Movies
Duplas com P-valor ruim:  Sports Movies e Faith & Spirituality
Duplas com P-valor bom:  Sports Movies e Anime Features
Duplas com P-valor bom:  Sports Movies e Movies
Duplas com P-valor bom:  Classic Movies e LGBTQ Movies
Duplas com P-valor ruim:  Classic Movies e Cult Movies
Duplas com P-valor bom:  Classic Movies e Faith & Spirituality
Duplas com P-valor bom:  Classic Movies e Anime Features
Duplas com P-valor bom:  Classic Movies e Movies
Duplas com P-valor bom:  LGBTQ Movies e Cult Movies
Duplas com P-valor ruim:  LGBTQ Movies e Faith & Spirituality
Duplas com P-valor ruim:  LGBTQ Movies e Anime Features
Duplas com P-valor bom:  LGBTQ Movies e Movies
Duplas com P-valor bom:  Cult Movies e Faith & Spirituality
Duplas com P-valor bom:  Cult Movies e Anime Features
Duplas com P-valor bom:  Cult Movies e Movies
Duplas com P-v